# Preprocessing our demo data

In [10]:
# -*- coding: utf-8 -*- #to be able to process arabic
import pandas as pd
import re
from camel_tools.utils.dediac import dediac_ar
from camel_tools.utils.charsets import AR_LETTERS_CHARSET
from camel_tools.utils.normalize import normalize_alef_maksura_ar
from camel_tools.utils.normalize import normalize_alef_ar
from camel_tools.utils.normalize import normalize_teh_marbuta_ar
from camel_tools.utils.normalize import normalize_unicode
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.morphology.database import MorphologyDB
from camel_tools.morphology.analyzer import Analyzer
from camel_tools.disambig.mle import MLEDisambiguator
mle_pretrained = MLEDisambiguator.pretrained()

First, let's load our data and investigate it.

In [2]:
data = pd.read_csv("Data\QA.csv")
data.head()

,Q_,A_
0,ايه هي الأنشطة التجارية اللي بتحتاج تصريح,1- تطوير وإنتاج وبيع أجهزة تشفير البيانات؛\n2-...
1,\nبنسجل الشركة فين؟,التسجيل بيبقى في مكتب السجل التجاري حسب الموقع...
2,لازم يكون شركتي ليها مقر؟,القانون الزم مؤسس الشركة انو يكون في مقر للشرك...
3,\nاعرف ازاي اسم شركتي مستخدم قبل كده ولا لا؟,في شهادة اسمها ” شهادة عدم الالتباس” بيتم استخ...
4,\nيعني أيه شركة ذات مسئولية محدودة؟,الشركات ذات المسئولية المحدودة:\n\nهي شركة عدد...


In [3]:
stopwords = []
with open('Data/arabic-stop-words-master/list.txt', encoding='utf8') as file:
    for line in file:
        stopwords.append(line.strip())
print(stopwords)
print(type(stopwords))   

['،', 'ء', 'ءَ', 'آ', 'آب', 'آذار', 'آض', 'آل', 'آمينَ', 'آناء', 'آنفا', 'آه', 'آهاً', 'آهٍ', 'آهِ', 'أ', 'أبدا', 'أبريل', 'أبو', 'أبٌ', 'أجل', 'أجمع', 'أحد', 'أخبر', 'أخذ', 'أخو', 'أخٌ', 'أربع', 'أربعاء', 'أربعة', 'أربعمئة', 'أربعمائة', 'أرى', 'أسكن', 'أصبح', 'أصلا', 'أضحى', 'أطعم', 'أعطى', 'أعلم', 'أغسطس', 'أفريل', 'أفعل به', 'أفٍّ', 'أقبل', 'أكتوبر', 'أل', 'ألا', 'ألف', 'ألفى', 'أم', 'أما', 'أمام', 'أمامك', 'أمامكَ', 'أمد', 'أمس', 'أمسى', 'أمّا', 'أن', 'أنا', 'أنبأ', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'أنتِ', 'أنشأ', 'أنه', 'أنًّ', 'أنّى', 'أهلا', 'أو', 'أوت', 'أوشك', 'أول', 'أولئك', 'أولاء', 'أولالك', 'أوّهْ', 'أى', 'أي', 'أيا', 'أيار', 'أيضا', 'أيلول', 'أين', 'أيّ', 'أيّان', 'أُفٍّ', 'ؤ', 'إحدى', 'إذ', 'إذا', 'إذاً', 'إذما', 'إذن', 'إزاء', 'إلى', 'إلي', 'إليكم', 'إليكما', 'إليكنّ', 'إليكَ', 'إلَيْكَ', 'إلّا', 'إمّا', 'إن', 'إنَّ', 'إى', 'إياك', 'إياكم', 'إياكما', 'إياكن', 'إيانا', 'إياه', 'إياها', 'إياهم', 'إياهما', 'إياهن', 'إياي', 'إيهٍ', 'ئ', 'ا', 'ا?', 'ا?ى', 'االا', 'االتى', 'اب

In [4]:
data.iloc[0][1]

'1- تطوير وإنتاج وبيع أجهزة تشفير البيانات؛\n2- إنشاء وتصميم وإصلاح والتخلص في صناعة الطيران؛\n3- صنع الأسلحة، المتفجرات، والأجهزة الكيميائية.\n4- إطفاء الحرائق الصناعية في الطبيعة، في المباني السكنية والصناعية، إذا كان هذا لا ينطبق على المساعدة الطوعية\n5- صيانة معدات السلامة من الحرائق.\n6- تصنيع الأدوية والمؤثرات العقلية والمخدرات والمشروبات الكحولية لخدمة المعدات الطبية؛\nالهندسة الوراثية، واستخدام الفيروسات التي تؤثر على كائن حي.\n7- نقل الأشخاص والبضائع وتحميل وتفريغ المواد الخطرة والسحب؛\n8- نقل الأشخاص، إذا كان هناك أكثر من 8 ركاب؛\n9- التخلص من المواد الخطرة على النفايات المحيطة وتخزينها؛\n10- تنظيم المقامرة\n11- حماية الأجسام\n12- معالجة وتخزين وبيع المعادن الحديدية وغير الحديدية؛\n13- توفير الاتصالات؛\n14- تطوير لمبيعات البرمجيات ومنتجات الفيديو؛\n15- التعليم؛\n16-استكشاف الفضاء.\n17- البحث عن الموارد الطبيعية، واستكشاف الأراضي لهذا الغرض؛\n18- العلاج؛\n19- جمع القطع الأثرية في محلات الرهونات؛\n20- أنشطة الخبراء في الصناعة؛\n21- العمل مع المتفجرات.'

In [5]:
data.shape

(24, 2)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q_      23 non-null     object
 1   A_      23 non-null     object
dtypes: object(2)
memory usage: 512.0+ bytes


### 1) Cleaning our data: 
- Remove URLS
- Mentions
- Emotions
- Symbols
- Diacritical marks
- Shapes
- Unwanted Punctuation (basicly anything that is not an arabic word or a number).

In [7]:
def clean_text(text):
    sentence_ar_dediac = dediac_ar(text) #dediacritization using camel tools
    sentence_ar_dediac = re.sub(r"\S*https?:\S*|@\S+", "", sentence_ar_dediac) #remove URLs and mentions even if they are in paranthesis or brackets
    no_punc = ""
    for char in sentence_ar_dediac:
        if char in (list(AR_LETTERS_CHARSET)+[" "]): #removing any thing that is not an arabic letter
            no_punc = no_punc + char
    return no_punc

After Cleaning,
### 2) Normalizing our data
- Orthographic Normalization
- Unicode Normalization

In [8]:
def normalize_text(text):
    cleaned_text = normalize_alef_maksura_ar(text)
    cleaned_text = normalize_alef_ar(cleaned_text)
    cleaned_text = normalize_teh_marbuta_ar(cleaned_text)
    cleaned_text = normalize_unicode(cleaned_text)
    
    return cleaned_text

### 3) Data Enrichement
- Tokenization
- Removing stop words
- Morphology Analysis
- Lemmatization

In [11]:
def enrichement(text, stopwords=stopwords, mle_pretrained=mle_pretrained):
    tokens = simple_word_tokenize(text)
    tokenized = []
    for token in tokens:
        if token not in stopwords:
            tokenized.append(token)     
    disambig = mle_pretrained.disambiguate(tokenized)
    lemmas = [d.analyses[0].analysis['lex'] for d in disambig]
    return lemmas

In [12]:
def preprocess_text(text, clean_text=clean_text, normalize_text=normalize_text, enrichement=enrichement):
    '''
    Inputs:
    text: string in which it will preprocess.
    
    Outputs:
    cleaned_text: the string after it was processed.
    '''
    
    #let's first start with cleaning 
    cleaned = clean_text(text)
            
    # now, for the normalization part
    normalized = normalize_text(cleaned)
    
    #enrichement
    enriched = enrichement(normalized)
    
    return enriched

In [13]:
def preprocess_data(data, preprocess_text=preprocess_text):
    '''
    Inputs:
    data: is the dataframe we want to apply our function on
    
    Outputs:
    data: the dataframe after applying the cleaning and normalization on each cell.
    
    '''
    for row in range(data.shape[0]):
        for column in range(data.shape[1]):
            for_cleaning = str(data.iloc[row][column])
            data.iloc[row][column] = preprocess_text(for_cleaning)
    return data
    

In [14]:
data = preprocess_data(data)
data.head()

,Q_,A_
0,"[إِيه, نَشاط, تِجارِيّ, لَيّ, بتحتاج, تَصْرِيح]","[تَطْوِير, إِنْتاج, بيع, جِهاز, تشفير, بَيان, ..."
1,"[بنسجل, شَرِكَة, يِن]","[تَسْجِيل, بيبقي, مَكْتَب, سِجِلّ, تِجارِيّ, م..."
2,"[لازِم, شَرِكَة, لَيّ, مَقَرّ]","[قانُون, لَزِم, مُؤَسِّس, شَرِكَة, نَوَى, مَقَ..."
3,"[عَرَف, ازاي, ٱِسْم, شَرِكَة, مُسْتَخْدَم, كَدّ]","[شَهادَة, ٱِسْم, شَهادَة, ٱِلْتِباس, بَيَّت, ٱ..."
4,"[عَنَى, إِيه, شَرِكَة, مَسْؤُول, مَحْدُود]","[شَرِكَة, مَسْؤُولِيَّة, المحدودههي, شَرِكَة, ..."


In [15]:
data.iloc[0][1]

['تَطْوِير',
 'إِنْتاج',
 'بيع',
 'جِهاز',
 'تشفير',
 'بَيان',
 'إِنْشاء',
 'تَصْمِيم',
 'إِصْلاح',
 'تَخَلُّص',
 'صِناعَة',
 'طَيَران',
 'صَنْع',
 'سِلاح',
 'مُتَفَجِّر',
 'جِهاز',
 'كِيمِيائِيّ',
 'إِطْفاء',
 'حَرِيق',
 'صِناعِيّ',
 'طَبِيعَة',
 'مَبْنَى',
 'سَكَنِيّ',
 'صِناعِيّ',
 'ٱِنْطَبَق',
 'مُساعَدَة',
 'طَوْعِيّ',
 'صِيانَة',
 'مُعَدّات',
 'سَلامَة',
 'حَرِيق',
 'تَصْنِيع',
 'دَواء',
 'مُؤَثِّر',
 'عَقْلِيَّة',
 'مُخَدِّر',
 'مَشْرُوب',
 'كُحُولِيّ',
 'خِدْمَة',
 'مُعَدّات',
 'الطبيهالهندسه',
 'وِراثِيّ',
 'ٱِسْتِخْدام',
 'فَيْرُوس',
 'أَثَّر',
 'كائِن',
 'حَيّ',
 'نَقْل',
 'شَخْص',
 'بِضاعَة',
 'تَحْمِيل',
 'تَفْرِيغ',
 'مادَّة',
 'خَطِر',
 'سَحْب',
 'نَقْل',
 'شَخْص',
 'راكِب',
 'تَخَلُّص',
 'مادَّة',
 'خَطِر',
 'نُفايَة',
 'مُحِيط',
 'تَخْزِين',
 'تَنْظِيم',
 'مُقامَرَة',
 'حِمايَة',
 'جِسْم',
 'مُعالَجَة',
 'تَخْزِين',
 'بيع',
 'مَعادِن',
 'حَدِيدِيّ',
 'غَيْر',
 'حَدِيدِيّ',
 'تَوْفِير',
 'ٱِتِّصال',
 'تَطْوِير',
 'مَبِيع',
 'بَرْمَجِيّات',
 'مُنْتَج',
 'فِيدْيُو',
 'الت